# 프로젝트: 얼굴을 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

패키지 설치

> pip install --user mediapipe

## 얼굴에 네모박스 치기

In [1]:
import cv2
import mediapipe as mp

# MediaPipe 얼굴 인식 초기화
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils #얼굴의 특징을 그리기 위한 drawing_utils모듈을 사용

# 비디오 캡처
cap=cv2.VideoCapture('face_video.mp4')

# 비디오 프레임 처리
#model_selection=0or1(0은 2m이내의 근거리, 1은 5m이내의 원거리 )
#min_detection_confidence=0.5는 50%이상의 신뢰도로 여기에 얼굴이 있는 것 같아. 라고 생각함.(임계치 개념)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #제대로 동영상이 열렸는가?
    while cap. isOpened():
        success, image=cap.read()
        if not success:
            break
            
        image.flags.writeable=False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#cv2.cvtColor: 프레임의 색상을 BGR에서 RGB로 변환
        results=face_detection.process(image) #변환된 이미지에서 얼굴을 인식
        
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #검출된 얼굴이 있다면? 그림 그리기
        if results.detections:
            #6개 특징: 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀 구슬점, 이주(귓구멍 앞부분))
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection) #인식된 얼굴에 대해 시각적인 표시
                #print(detection)
        
        #동영상 보여주는 부분
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5)) #처리된 이미지를 화면에 표시
        
        #q누르면 while문 종료
        if cv2.waitKey(1) == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

## 눈, 코 인식해서 도형 그리기

In [14]:
import cv2
import mediapipe as mp

# MediaPipe 얼굴 인식 초기화
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils #얼굴의 특징을 그리기 위한 drawing_utils모듈을 사용

# 비디오 캡처
cap=cv2.VideoCapture('face_video.mp4')

# 비디오 프레임 처리
#model_selection=0or1(0은 2m이내의 근거리, 1은 5m이내의 원거리 )
#min_detection_confidence=0.5는 50%이상의 신뢰도로 여기에 얼굴이 있는 것 같아. 라고 생각함.(임계치 개념)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #제대로 동영상이 열렸는가?
    while cap. isOpened():
        success, image=cap.read()
        if not success:
            break
            
        image.flags.writeable=False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#cv2.cvtColor: 프레임의 색상을 BGR에서 RGB로 변환
        results=face_detection.process(image) #변환된 이미지에서 얼굴을 인식
        
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #검출된 얼굴이 있다면? 그림 그리기
        if results.detections:
            #6개 특징: 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀 구슬점, 이주(귓구멍 앞부분)) 추출
            for detection in results.detections:
                #흰색 네모박스+6개의 포인트에 빨간 점 찍기
                mp_drawing.draw_detection(image, detection) #인식된 얼굴에 대해 시각적인 표시
                print(detection) #추출된 특징을 print로 확인하기
                
                #특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye=keypoints[0] #오른쪽 눈
                left_eye=keypoints[1] #왼쪽 눈
                nose_tip=keypoints[2] #코 끝부분 
                
                #이미지의 크기 정보를 가지고 좌표 계산하기
                h,w,_=image.shape #height, width, channel: 이미지로부터 세로, 가로 크기 가져옴
                right_eye=(int(right_eye.x*w), int(right_eye.y*h)) #이미지 내의 실제 좌표(x,y)
                left_eye=(int(left_eye.x*w), int(left_eye.y*h))
                nose_tip=(int(nose_tip.x*w), int(nose_tip.y*h))
                
                #양 눈에 동그라미 그리기 
                cv2.circle(image, right_eye, 50, (255,0,0), 10, cv2.LINE_AA) #파란색 동그라미
                cv2.circle(image, left_eye, 50, (0,255,0), 10, cv2.LINE_AA)
                
                #코에 동그라미 그리기
                cv2.circle(image, nose_tip, 30, (0,255,255), 10, cv2.LINE_AA) #빨간색 동그라미 그리기
                
        #동영상 보여주는 부분
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5)) #처리된 이미지를 화면에 표시
        
        #q누르면 while문 종료
        if cv2.waitKey(1) == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

label_id: 0
score: 0.9720634
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.33977893
    ymin: 0.27596474
    width: 0.16592523
    height: 0.29497838
  }
  relative_keypoints {
    x: 0.39748585
    y: 0.35070306
  }
  relative_keypoints {
    x: 0.47279006
    y: 0.3487131
  }
  relative_keypoints {
    x: 0.44714963
    y: 0.41272855
  }
  relative_keypoints {
    x: 0.44401816
    y: 0.4775529
  }
  relative_keypoints {
    x: 0.33800027
    y: 0.39432424
  }
  relative_keypoints {
    x: 0.49792442
    y: 0.3847406
  }
}

label_id: 0
score: 0.96794623
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.331663
    ymin: 0.27528393
    width: 0.16455492
    height: 0.2925421
  }
  relative_keypoints {
    x: 0.38922748
    y: 0.3448243
  }
  relative_keypoints {
    x: 0.4635287
    y: 0.33934838
  }
  relative_keypoints {
    x: 0.4386563
    y: 0.39772916
  }
  relative_keypoints {
    x: 0.4356684
    y: 0.4

label_id: 0
score: 0.94844896
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.23424983
    ymin: 0.26762414
    width: 0.17349267
    height: 0.3084315
  }
  relative_keypoints {
    x: 0.30734906
    y: 0.34604418
  }
  relative_keypoints {
    x: 0.37730685
    y: 0.33845848
  }
  relative_keypoints {
    x: 0.36380604
    y: 0.40184718
  }
  relative_keypoints {
    x: 0.35900694
    y: 0.47414494
  }
  relative_keypoints {
    x: 0.23560207
    y: 0.40188283
  }
  relative_keypoints {
    x: 0.39272028
    y: 0.3768499
  }
}

label_id: 0
score: 0.94652617
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.22508496
    ymin: 0.26388824
    width: 0.17389944
    height: 0.30915433
  }
  relative_keypoints {
    x: 0.300436
    y: 0.34934163
  }
  relative_keypoints {
    x: 0.37047806
    y: 0.3338334
  }
  relative_keypoints {
    x: 0.36125347
    y: 0.4025452
  }
  relative_keypoints {
    x: 0.35712618
    y

label_id: 0
score: 0.906406
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.2249752
    ymin: 0.23358452
    width: 0.17313832
    height: 0.30780184
  }
  relative_keypoints {
    x: 0.29873332
    y: 0.31548774
  }
  relative_keypoints {
    x: 0.36843553
    y: 0.30355144
  }
  relative_keypoints {
    x: 0.35697868
    y: 0.37216282
  }
  relative_keypoints {
    x: 0.35182428
    y: 0.44304788
  }
  relative_keypoints {
    x: 0.22506957
    y: 0.370687
  }
  relative_keypoints {
    x: 0.38057926
    y: 0.3392033
  }
}

label_id: 0
score: 0.93205184
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.23184146
    ymin: 0.24167389
    width: 0.16531561
    height: 0.2938949
  }
  relative_keypoints {
    x: 0.30353403
    y: 0.31804568
  }
  relative_keypoints {
    x: 0.37044924
    y: 0.30103576
  }
  relative_keypoints {
    x: 0.36308634
    y: 0.36486775
  }
  relative_keypoints {
    x: 0.35951874
    y:

label_id: 0
score: 0.95372486
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.3096927
    ymin: 0.26659775
    width: 0.17591187
    height: 0.31273293
  }
  relative_keypoints {
    x: 0.37258947
    y: 0.36019826
  }
  relative_keypoints {
    x: 0.44294924
    y: 0.35702926
  }
  relative_keypoints {
    x: 0.4182412
    y: 0.43924618
  }
  relative_keypoints {
    x: 0.41595206
    y: 0.49716234
  }
  relative_keypoints {
    x: 0.3174545
    y: 0.38017488
  }
  relative_keypoints {
    x: 0.4720121
    y: 0.3684308
  }
}

label_id: 0
score: 0.93666285
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.32601047
    ymin: 0.26342553
    width: 0.17239061
    height: 0.30647212
  }
  relative_keypoints {
    x: 0.37046057
    y: 0.36741287
  }
  relative_keypoints {
    x: 0.44119376
    y: 0.3505051
  }
  relative_keypoints {
    x: 0.40892413
    y: 0.4411041
  }
  relative_keypoints {
    x: 0.4149235
    y: 

label_id: 0
score: 0.9605235
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.32592985
    ymin: 0.29023838
    width: 0.1898332
    height: 0.33748168
  }
  relative_keypoints {
    x: 0.37843803
    y: 0.3968929
  }
  relative_keypoints {
    x: 0.46167362
    y: 0.3984781
  }
  relative_keypoints {
    x: 0.42150655
    y: 0.48934454
  }
  relative_keypoints {
    x: 0.42199016
    y: 0.5449651
  }
  relative_keypoints {
    x: 0.33155733
    y: 0.40127343
  }
  relative_keypoints {
    x: 0.51014274
    y: 0.40022212
  }
}

label_id: 0
score: 0.96138984
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.32321748
    ymin: 0.28597313
    width: 0.1884205
    height: 0.33496988
  }
  relative_keypoints {
    x: 0.37943357
    y: 0.38456547
  }
  relative_keypoints {
    x: 0.46089405
    y: 0.39534348
  }
  relative_keypoints {
    x: 0.42000628
    y: 0.48062146
  }
  relative_keypoints {
    x: 0.41648364
    y